# Welcome to TQ42

## Example of Multilayer Perceptron (MLP)

In [ ]:
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto
from tq42.algorithm import (
    AlgorithmProto, 
    OptimProto, 
    LossFuncProto, 
    DatasetStorageInfoProto, 
    MLTrainInputsProto,
    TSMLPTrainMetadataProto, 
    TSMLPTrainParametersProto,
    ActFuncProto
)
from google.protobuf.json_format import MessageToDict

# Create a client and selecting org, proj and exp

In [ ]:
with TQ42Client() as client:
    org_sample = list_all_organizations(client=client)[0] 
    proj_sample = list_all_projects(client=client, organization_id=org_sample.id)[0]
    exp_sample = list_all_experiments(client=client, project_id=proj_sample.id)[-1]

In [ ]:
    print(f"Running experiment within: Org {org_sample.id}, Proj {proj_sample.id} and Exp {exp_sample.id}`")

# Set parameters for the train

In [ ]:
    parameters_train = MessageToDict(TSMLPTrainMetadataProto(
        parameters=TSMLPTrainParametersProto(
            input_width=1,
            label_width=1,
            dim_list=[30, 45, 60],
            act_func=ActFuncProto.SIGMOID,
            dropout=True,
            dropout_p=0.5,
            bn=True,
            num_epochs=5,
            batch_size=20,
            learning_rate=0.05,
            optim=OptimProto.ADAM,
            loss_func=LossFuncProto.MSE,
            time_column="Time",
            target_column="Power, kW"
        ),
        inputs=MLTrainInputsProto(
            data=DatasetStorageInfoProto(storage_id="ENTER_DATASET_STORAGE_ID_HERE")
        )
    ), preserving_proto_field_name=True)

# Train the model

In [ ]:
    run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.TS_MLP_TRAIN, 
        experiment_id=exp_sample.id,
        compute=HardwareProto.SMALL, 
        parameters=parameters_train
    )
    
    print(run.data)

# Import functions for the evaluate

In [ ]:
from tq42.algorithm import (
    ModelStorageInfoProto,
    TSEvalInputsProto,
    TSMLPEvalMetadataProto,
    TSMLPEvalParametersProto
) 

# Set parameters for the evaluate

In [ ]:
params_eval =  MessageToDict(TSMLPEvalMetadataProto(
    parameters=TSMLPEvalParametersProto(
        input_width=1,
        label_width=1,
        dim_list=[30, 45, 60],
        act_func=ActFuncProto.SIGMOID,
        dropout=True,
        dropout_p=0.5,
        bn=True,
        time_column="Time"
    ),
    inputs=TSEvalInputsProto(
        model=ModelStorageInfoProto(storage_id="MODEL_STORAGE_BUCKET_ID"),
        data=DatasetStorageInfoProto(storage_id="DATA_STORAGE_BUCKET_ID")
    )
), preserving_proto_field_name=True)

# Evaluate the model

In [ ]:
with TQ42Client() as client:
    run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.TS_MLP_EVAL,
        experiment_id=exp_sample.id,
        compute=HardwareProto.SMALL,
        parameters=params_eval
    )
    print(run.data)